Auteur: Michaël Leroy

Ajout des coordonnées de la Mairie de chaque commune au decoupage administratif


[+] Méthodologie

    - Récupération des osmid des communes pour eviter le 'telescopage' des objets dans les graphs
    - Récupération du point central de la commune en remplacement des centroids des polygones:
        * centré sur une zone habitée et du coup route qui passe pas loin


[-] Long mais fait qu'une seule fois et réutilisable pour toutes les cartes    cartes de l'app les marqueurs seront placés en zone habitée (pas loin de la mairie à la campagne)


In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = False

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

In [ ]:
AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

In [ ]:
def missing(df):
    return pd.DataFrame({'column_name': df.columns,
                         'percent_missing': df.isnull().sum() * 100 / len(df)
                         }).sort_values(by='percent_missing',ascending=True)

## Get communes

In [ ]:
# # Pour charger le fichier utilisé dans le workflow 

# file_name = 'decoupage_administratif'
# ext = '.json'
# url ='https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581'
# # temp_path = 'temp_unzip'


# try: 
#     print('Loading data from local file...')
#     communes = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
# except:
#     print('Loading data from url...')

#     s = requests.get(url).content
#     communes = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
#     print('Saving data to local file...')
#     communes.to_feather(os.path.join(data_path,file_name + '.feather'))

# communes.info()




In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'
communes = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
communes.head(2)

In [ ]:
import requests
from requests.adapters import HTTPAdapter
import urllib3
from urllib3 import Retry

session = requests.Session()
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
session.headers.update(headers)
adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
session.mount("http://", adapter)
session.mount("https://", adapter)

def get_(url):
    response = session.get(url.strip())
    return response.json()
# GET COORDINATES FROM CITY NAME

def get_place(city, county, country):
    url  =  'https://nominatim.openstreetmap.org/search?'
    # url += f'q={adress}%2C+'
    url += f'city={city}&'
    url += f'country={country}&'
    url += f'county={county}&'


    url +=  'format=json' #&polygon_geojson=0'
    return get_(url)


In [ ]:
communes.query('libgeo == "Couvrot"')

In [ ]:
get_place('Couvrot','Marne',  'France')

In [ ]:
# nodes = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_with_known_tmja_mthd1.pkl'))
# nodes.head(1)

In [ ]:
#	codgeo	dep	reg	xcl2154	ycl2154	libgeo	geometry
#0	01001	01	84	848241	6563021	L'Abergement-Clémenciat	POLYGON ((4.90497 46.16079, 

# from shapely import Point
# ox.projection.project_geometry(
#     Point(848241,6563021) , 
#     crs=2154, to_crs=4326, to_latlong=False)

In [ ]:
Num2name = {v: k for k, v in Name2num.items()}
communes['osmid'] = None
communes['x'] = None
communes['y'] = None
# communes['point'] = None
errors = []
total = communes.shape[0]
for n, row in communes.iterrows():
    print(f'{(n + 1) / total:.2%} ', end=' '*50 + '\r')
    # print(row,'dep name:'+Num2name[row.dep])
    try:
        data = get_place(row.libgeo, Num2name[row.dep], 'france')[0]
        communes.loc[n,'osmid'] = data['osm_id']
        communes.loc[n,'y'] = data['lon']
        communes.loc[n,'x'] = data['lat']
    except :     
        errors.append(n)
    # break
    
print(f'Errors: {len(errors)}')
communes.sample(10)

In [19]:
data_path = './datas'
file_name = 'decoupage_administratif'
communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

In [22]:
temp = communes.dropna(subset='osmid')
temp.osmid.astype(int).astype(str)

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


0          142240
1          147124
2          145704
3          148821
4          146573
           ...   
34820      450627
34821      534505
34822      138530
34823      374081
34953    12124452
Name: osmid, Length: 34822, dtype: object

In [23]:
# Mappers to link osmids to commune names

temp = communes.dropna(subset='osmid')

# Dict to map osmid  -> communes names
joblib.dump(
    dict(zip(temp.osmid.astype(int).astype(str), temp.libgeo.str.upper())),
    os.path.join(data_path,f'[{CFG.area_name}]mapper_osmid_to_libgeo.pkl')
)

# Dict to map communes names  -> osmid 
joblib.dump(
    dict(zip(temp.libgeo.str.upper(), temp.osmid.astype(int).astype(str))),
    os.path.join(data_path,f'[{CFG.area_name}]mapper_libgeo_to_osmid.pkl')
)

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


['./datas\\[france]mapper_libgeo_to_osmid.pkl']

# Save 

In [ ]:
communes.to_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'
communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

In [ ]:
communes['tmp'] = communes.x

In [ ]:
communes.x = communes.y

In [ ]:
communes.y = communes.tmp

In [ ]:
communes.drop('tmp', axis=1, inplace=True)

In [ ]:
from shapely import box

box(-10,40,10,60)

keys= Num2name.keys()
to_fix = communes.loc[errors].query("dep in @keys")
to_fix

seulement 4 communes à localiser sur maps et à corriger manuellement avec Nominatim

https://nominatim.openstreetmap.org/ui/search.html?q=Vimartin-sur-Orthe+france+metropolitaine&countrycodes=fr

    index 5228  codgeo 16052   Bors (Canton de Tude-et-Lavalette) 45.34241 0.14665 OSMID 555489493
    index 5229  codgeo 16053   Bors (Canton de Charente-Sud) 45.32810  -0.19941    OSMID 125363211
    index 10916 codgeo 30014   Arpaillargues-et-Aureillac 44.000480, 4.368443      OSMID 9096959355
    index 34954 codgeo 53249   Vimartin sur ORTHE 48.21385  -0.21375               OSMID 41978755


------------------------------

In [ ]:
row = to_fix.iloc[2]
print(row.libgeo, Num2name[row.dep])
get_place(row.libgeo, Num2name[row.dep], 'france')[0]

In [ ]:
# Check if errors are fixable
errors_2 = []
total = communes.loc[errors].shape[0]
for n, row in communes.loc[errors].iterrows():
    print(f'{(n + 1) } ', end=' '*50 + '\r')
    # print(row,'dep name:'+Num2name[row.dep])
    try:
        data = get_place(row.libgeo, Num2name[row.dep], 'france')[0]
        print(data)
        communes.loc[n,'osmid'] = data['osm_id']
        communes.loc[n,'x'] = data['lon']
        communes.loc[n,'y'] = data['lat']
    except :     
        errors_2.append(n)
    # break
    
print(f'Errors: {len(errors_2)}')

# Fix errors

-------------------------------------

In [ ]:
***************** Fin du traitement *****************

## Get services publics

In [ ]:
data_path = './osm_datas'
file_name = 'datatourisme-place-20230529'

# tourism = gpd.read_file(os.path.join(data_path,file_name + '.csv'))
# tourism.to_feather(os.path.join(data_path,file_name + '.feather'))



tourism = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
display(tourism.info())
tourism.shape

In [ ]:
test = ox.geometries.geometries_from_address('larzicourt' + ', Metropolitan France',
                                        tags={'amenity':'townhall '}
                                        ) 
missing(test)

In [ ]:
test

In [ ]:
# TODO: rename file with []
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]boundaries.feather')
try:
  boundaries = gpd.read_feather(file)
except:
    print('Datas not found')
    boundaries = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      boundaries = pd.concat(
        [boundaries,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'boundary':'administrative'}, 
                                     which_result=1, buffer_dist=None)
                    #                  [['name', 
                    #            'geometry',
                    #   ]]
        ]
      )
    boundaries = boundaries[[
      'boundary','geometry','admin_level'
    ]]
    boundaries.to_feather(file)                                  

display(boundaries.info())
# tourism_tags['amenity'].value_counts()   
boundaries                                 

In [ ]:
test = ox.geometries.geometries_from_place('marne' + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
missing(test)

In [ ]:
file = os.path.join(CFG.data_path, f'[{CFG.area_name}]tourism_tags.feather')
try:
  tourism_tags = gpd.read_feather(file)
except:
    print('Datas not found')
    tourism_tags = gpd.GeoDataFrame()
    total = len(AREA)
    for n, area in enumerate(AREA):
      print(f'Datas retrieved for {area}: {n+1}/{total} ~ {(n+1) / total:.2%}')
      tourism_tags = pd.concat(
        [tourism_tags,
         ox.geometries.geometries_from_place(area + ', Metropolitan France',
                                        tags={'tourism': True}, 
                                     which_result=None, buffer_dist=None)
                                     [['tourism', 
                               'geometry',
                      ]]
        ]
      )
   
    tourism_tags.to_feather(file)                                  

display(tourism_tags.info())
display(tourism_tags['tourism'].value_counts())

del tourism_tags

In [ ]:
# TODO: get each commune Mairie coordinates

In [ ]:
# TODO: merge decoupage with osm_id and Mairies coordinates

In [ ]:
******* Fin du taritement des données **************


In [ ]:
# Add isom_id to communes decoupage, for graph building

import requests, io
data_path = './datas'
file_name = 'decoupage_administratif'
ext = '.json'
url ='https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581'


# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')

#         data = get_place_osmid('', row['libgeo'], 'France')

#         try:
#             gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         except:
#             gdf.loc[n, 'osm_id'] = None
#         try:
#             gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         except:
#             gdf.loc[n, 'place_id'] = None
#         try:
#             gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         except:
#             gdf.loc[n, 'place_rank'] = None
#         try:
#             gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         except:
#             gdf.loc[n, 'display_name'] = None
#         try:
#             gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#         except:
#             gdf.loc[n, 'importance'] = None
            
#     return gdf


# try: 
#     print('Loading data from local file...')
#     decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
    
# except:
#     print('Loading data from url...')

#     s = requests.get(url).content
#     decoupage = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# # Check if osm_id is present and complete gdf if needed
# if 'osm_id' not in decoupage.columns:
#     print('Adding osm feature (approx 5 hours to complete!!!)')
#     decoupage = get_osm_infos(decoupage)
#     print('Saving data to local file...')
#     decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

# display(decoupage.info())
# display(decoupage.head())

In [ ]:
data_path = './datas'
file_name = 'decoupage_administratif'
decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
decoupage.plot()

In [ ]:
# import requests
# from requests.adapters import HTTPAdapter
# import urllib3
# from urllib3 import Retry

# session = requests.Session()
# headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
# session.headers.update(headers)
# adapter = HTTPAdapter(max_retries=Retry(total=10, backoff_factor=1))
# session.mount("http://", adapter)
# session.mount("https://", adapter)

# def get_(url):
#     response = session.get(url.strip())
#     return response.json()
# # GET COORDINATES FROM CITY NAME

# def get_place_osmid(county,city, country):
#     url  =  'https://nominatim.openstreetmap.org/search?'
#     # url += f'q={adress}%2C+'
#     url += f'city={city}&'
#     url += f'country={country}&'
#     url += f'county={county}&'


#     url +=  'format=geojson'#&polygon_geojson=1'
#     return get_(url)


In [ ]:
# import concurrent.futures



# def get_osm_infos(gdf):
#     total = gdf.shape[0]
#     for i, (n, row) in enumerate(gdf.iterrows()):
#         print(f'{i/total:.2%}                   ', end='\r')
#         data = get_place_osmid('', row['libgeo'], 'France')
#         gdf.loc[n, 'osm_id'] = data['features'][0]['properties']['osm_id']
#         gdf.loc[n, 'place_id'] = data['features'][0]['properties']['place_id']
#         gdf.loc[n, 'place_rank'] = data['features'][0]['properties']['place_rank']
#         gdf.loc[n, 'display_name'] = data['features'][0]['properties']['display_name']
#         gdf.loc[n, 'importance'] = data['features'][0]['properties']['importance']
#     return gdf


# def multi_process_osmid(gdf, processes=4):
#     # Split the DataFrame into four equal chunks
#     num_chunks = processes
#     chunk_size = len(gdf) // num_chunks
#     gdf_chunks = [gdf[i:i+chunk_size].copy() for i in range(0, len(gdf), chunk_size)]

#     # Create a ThreadPoolExecutor to process chunks asynchronously
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Submit the processing function for each chunk
#         futures = [executor.submit(get_osm_infos, chunk) for chunk in gdf_chunks]

#         # Wait for all futures to complete
#         concurrent.futures.wait(futures)

#         # Get the results from the completed futures
#         results = pd.concat([future.result() for future in futures] )
#     return results       


In [ ]:
# decoupage = get_osm_infos(decoupage)

In [ ]:
# decoupage = multi_process_osmid(decoupage, processes=2)